In [1]:
def convert_files_test(sampling_file, command_file):

    if command_file.endswith("000"):
        print(command_file[-5:])

    #random.shuffle(out) #This will shuffle the rows
    with open(command_file, "r") as f:
        s = f.readlines()[0].split()

    if -0.01 < float(s[11]) < 0.01:
        return None
    
    if float(s[8]) < 0.05:
        return None

    return 1

In [2]:
from processing import *
import models
import torch

train_prop = 0.9
num_files = 100_000
data_dir = "../Data3"
max_batch_size = 128
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Define model
model1 = models.TransformerModel1()
model1.load_state_dict(torch.load("subset1.pth", map_location=torch.device(device)))
model1 = model1.to(device)
model1.eval()

X = [convert_files_test(data_dir + "/sampled_genotypes/sample_" + str(i), data_dir + "/commands/command_" + str(i)) for i in range(num_files)]
indices = [i for i in range(num_files) if X[i] is not None]

# Recreate test indices
torch.manual_seed(random_seed)
ind = int(train_prop * len(indices))
idx = torch.randperm(len(indices))
test_idx = idx[ind:].tolist()
test_indices = [indices[i] for i in test_idx]


X = [convert_files(data_dir + "/sampled_genotypes/sample_" + str(i), data_dir + "/commands/command_" + str(i)) for i in test_indices] 

C = [convert_command_file1(data_dir + "/commands/command_" + str(i)) for i in test_indices]
C = [c for x,c in zip(X,C) if x is not None]

X = [x for x in X if x is not None]

X = torch.tensor(X) - 1
X = X.reshape(-1, sample_width*num_chrom * 2)

C = torch.tensor(C)
C = C.reshape(-1, 11)

y = torch.tensor([i % 2 for i in range(1, X.shape[0] + 1)])

print(X.shape)




cuda
_1000
_2000
_3000
_4000
_5000
_6000
_7000
_8000
_9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
84000
90000
24000
25000
_9000
torch.Size([9830, 11000])


In [3]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1(X_batch)        

    weak_epistasis = (C[:,7] < C[:,10])
    weak_count = weak_epistasis.sum().item()

    # loss = criterion(y_pred, y.float())
    predictions = (y_pred >= 0.5).int()

    print()
    print("full set:")

    num_correct = (predictions == y).sum().item()
    print(f"Accuracy: {num_correct/y.shape[0]:0.4f}")

    print("\t0\t1")
    for i in range(2):
        called_0 = torch.sum((predictions == 0) & (y == i)).item()
        called_1 = torch.sum((predictions == 1) & (y == i)).item()
        sums = called_0 + called_1
        called_0 /= sums
        called_1 /= sums
        print(i, f"{called_0:.3f}" , f"{called_1:.3f}", sep='\t')
        
    called_epistasic = (predictions == 1)

    print()
    print("M and T evaluation")

    for i in range(8):
        minim = 0.1 + i/10
        maxum = 0.2 + i/10

        subset_0 = ((C[:,1] >= minim) & (C[:,1] < maxum))

        for j in range(10):

            minim = 100 + j*50
            maxum = 150 + j*50

            subset_1 = ((C[:,2] >= minim) & (C[:,2] < maxum))

            subset = (subset_0 & subset_1) # & called_epistasic)

            #Counting proportion of those called epistasis which are correct

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()

    print("epistrength and normstrength evaluation")

    for i in range(9):
        minim = 0.01 + i * 0.01
        maxum = 0.02 + i * 0.01

        subset_0 = ((C[:,7] >= minim) & (C[:,7] < maxum))

        for j in range(20):

            minim = -0.05  + j * 0.005
            maxum = -0.045 + j * 0.005

            subset_1 = ((C[:,10] >= minim) & (C[:,10] < maxum))

            subset = (subset_0 & subset_1) # & called_epistatic)

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()


full set:
Accuracy: 0.9413
	0	1
0	0.905	0.095
1	0.022	0.978

M and T evaluation
$$$#$$$$#$
$$####$###
#$####$#$#
$$$$##$#$$
$$$$$$$$$$
$###$#####
$########$
$$$$$#$$#$

epistrength and normstrength evaluation
00000000000000000000
00000000000000000000
00000000000000000000
00000000000000000000
$$$$#$#$0000####$$$$
$$#$###$0000####$$$$
$$$####$0000$#$$$$$#
$$#$###$0000$#$#$$$$
$$#####$0000$###$$$$



In [9]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1.evaltest(X_batch,8,128)        

    weak_epistasis = (C[:,7] < C[:,10])
    weak_count = weak_epistasis.sum().item()

    # loss = criterion(y_pred, y.float())
    predictions = (y_pred >= 0.5).int()

    print()
    print("full set:")

    num_correct = (predictions == y).sum().item()
    print(f"Accuracy: {num_correct/y.shape[0]:0.4f}")

    print("\t0\t1")
    for i in range(2):
        called_0 = torch.sum((predictions == 0) & (y == i)).item()
        called_1 = torch.sum((predictions == 1) & (y == i)).item()
        sums = called_0 + called_1
        called_0 /= sums
        called_1 /= sums
        print(i, f"{called_0:.3f}" , f"{called_1:.3f}", sep='\t')
        
    called_epistasic = (predictions == 1)

    print()
    print("M and T evaluation")

    for i in range(8):
        minim = 0.1 + i/10
        maxum = 0.2 + i/10

        subset_0 = ((C[:,1] >= minim) & (C[:,1] < maxum))

        for j in range(10):

            minim = 100 + j*50
            maxum = 150 + j*50

            subset_1 = ((C[:,2] >= minim) & (C[:,2] < maxum))

            subset = (subset_0 & subset_1) # & called_epistasic)

            #Counting proportion of those called epistasis which are correct

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()

    print("epistrength and normstrength evaluation")

    for i in range(9):
        minim = 0.01 + i * 0.01
        maxum = 0.02 + i * 0.01

        subset_0 = ((C[:,7] >= minim) & (C[:,7] < maxum))

        for j in range(20):

            minim = -0.05  + j * 0.005
            maxum = -0.045 + j * 0.005

            subset_1 = ((C[:,10] >= minim) & (C[:,10] < maxum))

            subset = (subset_0 & subset_1) # & called_epistatic)

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()


full set:
Accuracy: 0.9434
	0	1
0	0.905	0.095
1	0.019	0.981

M and T evaluation
#$$#$$$$#$
$#####$###
$#####$#$#
$$$$#$$$$$
$#$$#$$$$$
$###$#####
$$######$$
$$$$$#$$#$

epistrength and normstrength evaluation
00000000000000000000
00000000000000000000
00000000000000000000
00000000000000000000
$$$$##$#0000####$$$$
$$#$###$0000######$$
$$$#####0000$##$$$$#
$$#$###$0000$###$$$$
$$#####$0000$###$$$$



In [10]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1.evaltest(X_batch,600,128)        

    weak_epistasis = (C[:,7] < C[:,10])
    weak_count = weak_epistasis.sum().item()

    # loss = criterion(y_pred, y.float())
    predictions = (y_pred >= 0.5).int()

    print()
    print("full set:")

    num_correct = (predictions == y).sum().item()
    print(f"Accuracy: {num_correct/y.shape[0]:0.4f}")

    print("\t0\t1")
    for i in range(2):
        called_0 = torch.sum((predictions == 0) & (y == i)).item()
        called_1 = torch.sum((predictions == 1) & (y == i)).item()
        sums = called_0 + called_1
        called_0 /= sums
        called_1 /= sums
        print(i, f"{called_0:.3f}" , f"{called_1:.3f}", sep='\t')
        
    called_epistasic = (predictions == 1)

    print()
    print("M and T evaluation")

    for i in range(8):
        minim = 0.1 + i/10
        maxum = 0.2 + i/10

        subset_0 = ((C[:,1] >= minim) & (C[:,1] < maxum))

        for j in range(10):

            minim = 100 + j*50
            maxum = 150 + j*50

            subset_1 = ((C[:,2] >= minim) & (C[:,2] < maxum))

            subset = (subset_0 & subset_1) # & called_epistasic)

            #Counting proportion of those called epistasis which are correct

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()

    print("epistrength and normstrength evaluation")

    for i in range(9):
        minim = 0.01 + i * 0.01
        maxum = 0.02 + i * 0.01

        subset_0 = ((C[:,7] >= minim) & (C[:,7] < maxum))

        for j in range(20):

            minim = -0.05  + j * 0.005
            maxum = -0.045 + j * 0.005

            subset_1 = ((C[:,10] >= minim) & (C[:,10] < maxum))

            subset = (subset_0 & subset_1) # & called_epistatic)

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()


full set:
Accuracy: 0.9435
	0	1
0	0.905	0.095
1	0.018	0.982

M and T evaluation
#$$#$$$$#$
$#####$###
######$#$#
$$$$##$$$$
$#$$#$$$$$
$###$#####
$$######$$
$$$$$#$$#$

epistrength and normstrength evaluation
00000000000000000000
00000000000000000000
00000000000000000000
00000000000000000000
$$$$####0000####$$$#
$$#$###$0000######$$
$$$#####0000$##$$$$$
$$#$###$0000$###$$$$
$$#####$0000$###$$$$



In [4]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1(X_batch)

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.5f}")


Accuracy: 0.98861


In [5]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1.evaltest(X_batch, 4, 128)

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.5f}")

Accuracy: 0.98922


In [6]:
y_pred = model1.evaltest(X,16,128)
num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.5f}")


Accuracy: 0.98962


In [7]:
# y_pred = model1.evaltest(X,300,128)
# num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

# print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.5f}")

In [8]:
print(torch.cuda.is_available())

True
